In [1]:
import requests
import json
import config
import openai
import tqdm
import html
from datetime import datetime
from story_feedback import WorkItem, get_work_item

ADO_URL = f"https://dev.azure.com/"
ADO_URL += f"{config.DEVOPS_ORG}/" 
ADO_URL += f"_apis/wit/wiql"

api_version_param = {"api-version":config.API_VERSION}

class QueryExecutionError(Exception):
    """Exception raised for errors during query execution."""
    pass

def run_query(query, params={}):
    query_params = api_version_param | params

    payload = json.dumps({"query":query})
    response = requests.post(ADO_URL, headers=config.PRIMARY_HEADERS, data=payload, params=query_params)
    if response.status_code != 200:
        raise QueryExecutionError(f"Failed to execute query. Error: {response.json()}")

    return response.json() 

In [28]:
# Run a query to get the workitems from a particular project
work_items_by_project_query = """
SELECT
    [System.Id],
    [System.WorkItemType],
    [System.Title],
    [System.AssignedTo],
    [System.State],
    [System.Tags]
FROM workitems
WHERE
    [System.TeamProject] = '{project_name}'
    AND [System.WorkItemType] = 'User Story'
ORDER BY [System.Id]
"""

projects = ["F23 - P1", 'F23-P3', 'F23-P4', 'FinderSpa.com', 'PetPulse (F23-P2)']

def get_work_items_by_project(project_name):
    wiql_query = work_items_by_project_query.format(project_name=project_name)
    return run_query(query=wiql_query)['workItems']

work_items = get_work_items_by_project('Sample Project')
work_item_urls = [w.get("id") for w in work_items]

# Gather the descriptions and story points given the urls or ids

[7, 22, 25, 26, 31, 32, 230, 231, 308, 309, 311, 577, 689]


In [4]:
wi = get_work_item(7)
wi

In [15]:
def get_chat_feedback(work_items):
    '''
    Go to ChatGPT and ask for feedback on the user story.
    
    :returns: response from ChatGPT
    '''
    REQUEST = """The following are user stories and the story points associated with them.
                Consider them all, on the whole.  Do any of them standout as having too few
                or too many story points associated with them?  Are they consistent in sizing?
                Which ones, if any should be adjusted?  When you respond start with either GOOD: or NEEDS WORK:
    """
    conversation= [{"role":"system", "content":REQUEST},]
    for wi in work_items:
        conversation += [  
               {"role":"user", "content":wi.description},
               {"role":"user", "content":wi.acceptance_criteria},
               {"role":"user", "content":f"story points {wi.story_points}"}]
    
    response = openai.ChatCompletion.create(model='gpt-3.5-turbo',messages=conversation)
    message = response['choices'][0]['message']['content']
    return message

#wi_list = [get_work_item(i) for i in [7, 22, 25, 26, 31, 32, 230, 231, 308, 309, 311, 577, 689]]
#get_chat_feedback()


In [20]:
get_chat_feedback([wi_list[3]])

'GOOD: The story points for this user story seem appropriate. It involves the ability to view the release log and have it automatically updated, which could require some development effort. A story point of 8.0 indicates that it is a medium-sized user story.'